# Pandas로 국민청원 데이터 분석하기

## Pandas와 NumPy를 import해 옵니다.

In [ ]:
import pandas as pd
import numpy as np

## csv 데이터를 불러 옵니다.

In [ ]:
df = pd.read_csv('data/petition.csv', parse_dates=['start', 'end'])

## 읽어온 데이터가 몇 행 몇 열인지 봅니다.

In [ ]:
df.shape

## 일부 데이터 미리 보기
* 상단 5개의 데이터를 불러옵니다.

In [ ]:
df.head(5)

* 하단 3개의 데이터를 불러옵니다.

In [ ]:
df.tail(3)

## 결측치가 있는지 확인해 봅니다.

In [ ]:
df.isnull().sum()

## 데이터 요약하기
* 어떤 컬럼이 있고 어떤 타입인지 출력해 봅니다.

In [ ]:
df.info()

* 데이터 타입만 따로 뽑아 봅니다.

In [ ]:
df.dtypes

* 컬럼명만 따로 추출해 봅니다.

In [ ]:
df.columns

* 수치형 데이터에 대한 요약을 봅니다.

In [ ]:
df.describe()

* 카테고리(object) 형태의 데이터에 대한 요약을 봅니다.

In [ ]:
df.describe(include=np.object)

## 답변대상 청원 보기
20만건 이상 투표를 받으면 답변 대상 청원이 됩니다.<br/>20만건 이상 투표를 받은 청원의 갯수를 세어보세요.

In [ ]:
df_20 = df.loc[df['votes'] > 200000]
df_20.shape

In [ ]:
df.loc[df['votes'] > 1000 ].count()

In [ ]:
df_20.category.value_counts()

In [ ]:
df_20[['title', 'content']].head(3)

* 20만건 이상 투표를 받은 상위 5개의 청원을 head()를 통해 출력해 보세요.

In [ ]:
df_20_loc = df.loc[df.votes > 200000]
df_20_loc.sort_values(by='votes', ascending=False).head()

* 20만건 이상 투표를 받은 청원을 별도의 컬럼을 만들어 줍니다. 컬럼 이름은 `answer`로 합니다.

In [ ]:
df['answer'] = (df['votes'] > 200000) == 1

* df 데이터프레임의 크기를 다시 찍어 보세요. 컬럼 하나가 늘었나요?

In [ ]:
df.shape

In [ ]:
df.dtypes

* 새로 생성해 준 answer의 타입은 boolean 타입입니다. int로 변경해 보세요.

In [ ]:
df['answer'] = df['answer'].astype('int')

* 답변대상 청원중 아직 답변되지 않은 청원의 수를 계산해 보세요.

In [ ]:
df['answer_diff'] = df['answer'] - df['answered']
df['answer_diff'].sum()

## 답변 대상 청원 중 투표를 가장 많이 받은 것

In [ ]:
answered_df = df.loc[df['answer'] == 1]
answered_df.sort_values('votes', ascending=False).head(5)

## 어느 분야의 청원이 가장 많이 들어왔는지?
pandas의 value_counts로 특정 컬럼의 데이터를 그룹화하여 카운된 숫자를 볼 수 있습니다.<br/>
어느 분야의 청원이 가장 많이 들어왔는지 찾아보세요.

In [ ]:
category = pd.DataFrame(df['category'].value_counts()).reset_index()
category.columns = ['category', 'counts']
category

## 청원이 가장 많이 들어 온 날은 언제인지 정렬해 보세요.

In [ ]:
start_df = pd.DataFrame(df['start'].value_counts()).reset_index()
start_df.columns = ['start', 'counts']
start_df = start_df.sort_values('counts', ascending=False)
print('청원 집계: {}일'.format(start_df.shape[0]))
start_df.head()

## 피봇 테이블로 투표를 가장 많이 받은 분야를 찾아보세요.

In [ ]:
petitions_unique = pd.pivot_table(df, index=['category'], aggfunc=np.sum)
petitions_best = petitions_unique.sort_values(by='votes', \
                                              ascending=False).reset_index()
petitions_best

## 투표를 가장 많이 받은 날은 언제일까요?

In [ ]:
petitions_start = pd.pivot_table(df, index=['start'], aggfunc=np.sum)
votes_df = petitions_start.sort_values(by='votes', ascending=False)
votes_df.loc[petitions_start['votes'] > 350000]

## 청원을 많이 받은 날 VS 투표를 많이 받은 날에 대해 각각 상위 5개 목록을 추출해 봅니다. 
이때, title, content는 안 나와도 됩니다.

In [ ]:
# 두 개의 데이터프레임을 합치기 위해 인덱스를 생성한다.
votes_df = votes_df.reset_index()
hottest_day_df = start_df.merge(votes_df, on='start', how='left')
hottest_day_df.sort_values('counts', ascending=False)[:5]

In [ ]:
hottest_day_df.sort_values('votes', ascending=False)[:5]

## 시계열 데이터 보기
* 월별 청원수를 집계해 보세요.

In [ ]:
df['start_month'] = df['start'].dt.month
df['start_day'] = df['start'].dt.day
df['start_hour'] = df['start'].dt.hour
df['start_dow'] = df['start'].dt.dayofweek
df.shape

In [ ]:
df['start_month'].value_counts()

* 청원이 가장 많이 들어온 달은 언제인가요?
* 요일별 청원 수는 어떻게 되나요?

In [ ]:
crypto = df[( df.title.str.find('가상화폐') != -1 ) | ( df.content.str.find('가상화폐') != -1  )]
crypto.shape

In [ ]:
crypto.head()

In [ ]:
crypto['category'].value_counts()

## 특정 단어가 들어가는 청원을 찾아보세요.

In [ ]:
import re
p = r'.*(돌봄|아이|초등|보육).*'
care = df[df['title'].str.match(p) |
           df['content'].str.match(p, flags=re.MULTILINE)]
care.shape

## 위 분석 외에 각자 해보고 싶은 분석을 해보세요.